
## 🎮 Implementing Pentago game AI with Minimax Algorithm

**Objective:**  
Build a playable version of the Pentago game and implement an AI opponent using the Minimax algorithm.

---

# <span style="color: #3498db;">Minmax Algorithm</span>

In [1]:
import random
import numpy as np
from math import inf
import time
import pygame

pygame 2.6.1 (SDL 2.28.4, Python 3.12.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
weight_comp_heuristic = np.array([5,10,100, 1000])
weight_human_heuristic = np.array([-3,-8,-80,-800])
important_points = [[2, 2], [2, 3],[3, 2], [3, 3]]

In [3]:
class PentagoGame:
    def __init__(self, prunning, ui=False, print=False, depth=2):
        self.board = np.zeros((6, 6), dtype=int)
        self.current_player = 1
        self.ui = ui
        self.depth = depth
        self.nodes_visited = 0
        self.game_over = False
        self.result = None
        self.selected_block = None
        self.move_stage = 0  # 0: place piece, 1: select block, 2: rotate
        self.temp_piece = None
        self.print = print
        self.prunning = prunning
        self.total_time = 0
        self.total_nodes = 0
        if ui:
            pygame.font.init()
            self.font_large = pygame.font.SysFont("Arial", 36)
            self.font_small = pygame.font.SysFont("Arial", 24)
            self.screen = pygame.display.set_mode((800, 600))
            pygame.display.set_caption("Pygame Board")
            self.show_buttons = False
            self.buttons = {
                "rotate_cw": pygame.Rect(650, 200, 100, 50),
                "rotate_ccw": pygame.Rect(650, 300, 100, 50),
            }
            self.setup_controls()
            self.draw_board()


    def setup_controls(self):
        if self.show_buttons:
            pygame.draw.rect(self.screen, (144, 238, 144), self.buttons["rotate_cw"])   # Light Green
            pygame.draw.rect(self.screen, (173, 216, 230), self.buttons["rotate_ccw"])  # Light Blue

            self.screen.draw_text("CLOCKWISE", self.buttons["rotate_cw"].center)
            self.screen.draw_text("COUNTER-CLOCKWISE", self.buttons["rotate_ccw"].center)

    def hide_rotation_buttons(self):
        self.show_buttons = False

    def show_rotation_buttons(self):
        self.show_buttons = True

    def copy_board(self, board):
        return np.copy(board)

    def rotate_block(self, board, block, direction):
        row_start = (block // 2) * 3
        col_start = (block % 2) * 3
        sub = board[row_start : row_start + 3, col_start : col_start + 3]
        rotated = np.rot90(sub, 3 if direction == "cw" else 1)
        board[row_start : row_start + 3, col_start : col_start + 3] = rotated

    def get_possible_moves(self, board, player):
        moves = []
        for i in range(6):
            for j in range(6):
                if board[i][j] == 0:
                    for block in range(4):
                        for dir in ["cw", "ccw"]:
                            moves.append((i, j, block, dir))
        return moves

    def apply_move(self, board, move, player):
        new_board = self.copy_board(board)
        row, col, block, direction = move
        if new_board[row][col] != 0:
            return None
        new_board[row][col] = player
        self.rotate_block(new_board, block, direction)
        return new_board

    def check_winner(self, board):
        for i in range(6):
            for j in range(6):
                if board[i][j] == 0:
                    continue

                # Horizontal
                if j <= 1 and np.all(board[i, j : j + 5] == board[i][j]):
                    return board[i][j]

                # Vertical
                if i <= 1 and np.all(board[i : i + 5, j] == board[i][j]):
                    return board[i][j]

                # Diagonal
                if (
                    i <= 1
                    and j <= 1
                    and all(board[i + k][j + k] == board[i][j] for k in range(5))
                ):
                    return board[i][j]

                # Anti-diagonal
                if (
                    i <= 1
                    and j >= 4
                    and all(board[i + k][j - k] == board[i][j] for k in range(5))
                ):
                    return board[i][j]
        if np.all(board != 0):
            return 0
        return None
    
    def heuristic_for_arr(self, arr):
        cnt = [0,0,0]
        # cnt -1, 0, 1
        for elem in arr:
            cnt[elem + 1] += 1
        if cnt[2] > 0 and cnt[0] > 0:
            return 0
        if 2 <= cnt[0] <= 5:
            return weight_comp_heuristic[cnt[0] - 2]
        elif 2 <= cnt[2] <= 5:
            return weight_human_heuristic[cnt[2] - 2]
        return 0

    def heuristic(self, board):
        sum = 0
        for (i, j) in important_points:
                if board[i][j] == -1:
                    sum += 1
                elif board[i][j] == 1:
                    sum += -1
        for i in range(6):
            for j in range(2):
                horizontal = board[i, j:j+5]
                vertical = board[j:j+5, i]
                sum += self.heuristic_for_arr(horizontal)
                sum += self.heuristic_for_arr(vertical)

        for i in range(2):
            for j in range(2):
                diagonal = [board[i+k][j+k] for k in range(5)]
                anti_diagonal = [board[i+k][j+4-k] for k in range(5)]
                sum += self.heuristic_for_arr(diagonal)
                sum += self.heuristic_for_arr(anti_diagonal)
        return sum

    def max_value(self, board, depth,alpha, beta):
        self.nodes_visited += 1
        if depth == 0:
            return self.heuristic(board)
        max_val = -inf
        moves = self.get_possible_moves(board, -1)
        for move in moves:
            new_board = self.apply_move(board, move, -1)
            if new_board is None:
                continue
            val = self.min_value(new_board, depth-1, alpha, beta)
            max_val = max(val, max_val)
            alpha = max(alpha, val)
            if self.prunning:
                if alpha >= beta:
                    break
        return max_val

    def min_value(self, board, depth, alpha, beta):
        self.nodes_visited += 1
        if depth == 0:
            return self.heuristic(board)
        min_val = inf
        moves = self.get_possible_moves(board, 1)
        for move in moves:
            new_board = self.apply_move(board, move, 1)
            if new_board is None:
                continue
            val = self.max_value(new_board, depth-1, alpha, beta)
            min_val = min(min_val, val)
            beta = min(beta, val)
            if self.prunning:
                if alpha >= beta:
                    break
        return min_val

    def minimax(self, board, alpha, beta):
        return self.min_value(board, self.depth - 1, alpha, beta)

    def get_computer_move(self):
        start_time = time.time()
        best_move = None
        best_value = -inf
        
        moves = self.get_possible_moves(self.board, -1)
        if not moves:
            return None
        best_move = moves[0]
        alpha, beta = -inf, inf
        for move in moves:
            if self.game_over:
                break
            new_board = self.apply_move(self.board, move, -1)
            if new_board is None:
                continue

            try:
                value = self.minimax(new_board, alpha, beta)
                alpha = max(value, alpha)
            except:
                value = -inf

            if value > best_value:
                best_value = value
                best_move = move

        if self.print == True:
            self.total_nodes += self.nodes_visited
            self.total_time += time.time()-start_time
            print(f"Move took {time.time()-start_time:.2f}s, nodes visited: {self.nodes_visited}")
        self.nodes_visited = 0
        return best_move

    def draw_text(self, text, center_pos, max_width):
        font_size = 24
        font = pygame.font.Font(None, font_size)
        text_surface = font.render(text, True, (0, 0, 0))

        text_width = text_surface.get_width()
        if text_width > max_width:
            scale_factor = max_width / text_width
            new_font_size = int(font_size * scale_factor)
            font = pygame.font.Font(None, new_font_size)
            text_surface = font.render(text, True, (0, 0, 0))

        text_rect = text_surface.get_rect(center=center_pos)
        self.screen.blit(text_surface, text_rect)

    def draw_board(self):
        self.screen.fill((0, 0, 0))

        for i in range(6):
            for j in range(6):
                x0 = j * 100
                y0 = i * 100

                if self.board[i][j] == 1:
                    pygame.draw.circle(self.screen, (255, 0, 0), (x0 + 50, y0 + 50), 40)
                elif self.board[i][j] == -1:
                    pygame.draw.circle(self.screen, (0, 0, 255), (x0 + 50, y0 + 50), 40)

                pygame.draw.rect(self.screen, (255, 255, 255), (x0, y0, 100, 100), 1)

        for i in [3, 6]:
            pygame.draw.line(self.screen, (255, 255, 255), (0, i * 100), (600, i * 100), 3)  # Horizontal
            pygame.draw.line(self.screen, (255, 255, 255), (i * 100, 0), (i * 100, 600), 3)  # Vertical

        # Show rotation buttons if in move_stage 2
        if self.move_stage == 2:
            self.highlight_selected_block()
            self.show_rotation_buttons()

        if self.show_buttons:
            pygame.draw.rect(self.screen, (144, 238, 144), self.buttons["rotate_cw"])  # Light Green
            pygame.draw.rect(self.screen, (173, 216, 230), self.buttons["rotate_ccw"])  # Light Blue

            self.draw_text(
                "CLOCKWISE",
                self.buttons["rotate_cw"].center,
                self.buttons["rotate_cw"].width,
            )
            self.draw_text(
                "COUNTER-CLOCKWISE",
                self.buttons["rotate_ccw"].center,
                self.buttons["rotate_ccw"].width,
            )

    def click_handler(self, event):
        if self.game_over or self.current_player != 1:
            return

        x, y = event.pos
        if self.move_stage == 0:  # Place piece
            if x > 600:
                return  # clicks on control area
            col = x // 100
            row = y // 100
            if 0 <= row < 6 and 0 <= col < 6 and self.board[row][col] == 0:
                self.temp_piece = (row, col)
                self.board[row][col] = 1
                self.move_stage = 1
                self.draw_board()

        elif self.move_stage == 1:  # Select block
            if x > 600:
                return
            # which block was clicked
            block_x = 0 if x < 300 else 1
            block_y = 0 if y < 300 else 1
            self.selected_block = block_y * 2 + block_x
            self.move_stage = 2
            self.show_rotation_buttons()
            self.highlight_selected_block()

        elif self.move_stage == 2:  # Rotate
            if self.buttons["rotate_cw"].collidepoint(event.pos):
                self.apply_rotation("cw")
            if self.buttons["rotate_ccw"].collidepoint(event.pos):
                self.apply_rotation("ccw")

    def apply_rotation(self, direction):
        self.rotate_block(self.board, self.selected_block, direction)
        self.current_player = -1
        self.move_stage = 0
        self.selected_block = None
        self.temp_piece = None
        self.hide_rotation_buttons()
        self.draw_board()
        pygame.display.flip()
        self.check_game_over()
        pygame.time.delay(1000)
        self.play_computer_move()

    def highlight_selected_block(self):
        colors = [
            (255, 153, 153),
            (153, 255, 153),
            (153, 153, 255),
            (255, 255, 153),
        ]  # RGB colors

        row_start = (self.selected_block // 2) * 3
        col_start = (self.selected_block % 2) * 3

        pygame.draw.rect(
            self.screen,
            colors[self.selected_block],
            (col_start * 100, row_start * 100, 300, 300),
            5,
        )

    def play_computer_move(self):
        move = self.get_computer_move()
        if move and not self.game_over:
            new_board = self.apply_move(self.board, move, -1)
            if new_board is not None:
                self.board = new_board
                self.current_player = 1
                self.draw_board()
                pygame.display.flip()
                self.check_game_over()
            else:
                print("Invalid computer move!")

    def check_game_over(self):
        winner = self.check_winner(self.board)
        if winner is not None:
            self.game_over = True
            self.result = winner
            print("Game over! Result:", winner)
            if self.ui:
                self.show_game_over_message()

    def show_game_over_message(self):
        self.screen.fill((200, 200, 200))
        pygame.draw.rect(self.screen, (255, 255, 255), (100, 200, 500, 200))
        pygame.draw.rect(self.screen, (0, 0, 0), (100, 200, 500, 200), 3)

        result_text = f"Player {self.result} wins!" if self.result != 0 else "Draw!"
        text_surface = self.font_large.render(result_text, True, (255, 0, 0))
        self.screen.blit(text_surface, (250, 250))

        exit_text = self.font_small.render("Click anywhere to exit", True, (0, 0, 0))
        self.screen.blit(exit_text, (230, 350))
        pygame.display.flip()

    def play(self):
        if self.ui:
            running = True
            while running:
                for event in pygame.event.get():
                    if event.type == pygame.QUIT:
                        running = False
                    elif event.type == pygame.MOUSEBUTTONDOWN:
                        self.click_handler(event)
                self.draw_board()
                pygame.display.flip()
            pygame.quit()
            return self.result
        else:
            while not self.game_over:
                self.print_board()
                winner = self.check_winner(self.board)
                if winner is not None:

                    return winner

                if self.current_player == 1:
                    move = random.choice(self.get_possible_moves(self.board, 1))
                else:
                    move = self.get_computer_move()

                self.board = self.apply_move(self.board, move, self.current_player)
                self.current_player *= -1
            return self.result

    def print_board(self):
        if self.print == False:
            return
        print("-" * 25)
        for row in self.board:
            print(" ".join(f"{x:2}" for x in row))
        print("-" * 25)
    def get_total_time(self):
        return self.total_time
    def get_total_nodes(self):
        return self.total_nodes

### To Prune or Not?

---

Run 5 simulation games using:
- **Alpha-Beta Pruning**
- **No Pruning (Plain Minimax)**

For each approach, measure and compare:
- Average duration per game
- Average number of visited nodes per game
- Success probability (win rate) per player

In [4]:
numGames = 5
numWins, numTies, numLosses = 0, 0, 0
prunning = False
total_time = 0
total_nodes = 0
for i in range(numGames):
    game = PentagoGame(prunning, ui=False, print=True, depth=2)
    result = game.play()
    if result == -1:
        numWins += 1
    elif result == 0:
        numTies += 1
    else:
        numLosses += 1
    total_time += game.get_total_time()
    total_nodes += game.get_total_nodes()
    print(f"in game {i} out of {numGames} took {game.get_total_time()} seconds and {game.get_total_nodes()} nodes were visited")
print(f"{numWins} wins, {numTies} ties, {numLosses} losses, prunning {prunning}")

print(f"mean time is {total_time / numGames } seconds, mean visited nodes {total_nodes / numGames}, probility of success {numWins / numGames}")

-------------------------
 0  0  0  0  0  0
 0  0  0  0  0  0
 0  0  0  0  0  0
 0  0  0  0  0  0
 0  0  0  0  0  0
 0  0  0  0  0  0
-------------------------
-------------------------
 1  0  0  0  0  0
 0  0  0  0  0  0
 0  0  0  0  0  0
 0  0  0  0  0  0
 0  0  0  0  0  0
 0  0  0  0  0  0
-------------------------
Move took 10.80s, nodes visited: 76440
-------------------------
 1  0 -1  0  0  0
 0  0  0  0  0  0
 0  0  0  0  0  0
 0  0  0  0  0  0
 0  0  0  0  0  0
 0  0  0  0  0  0
-------------------------
-------------------------
 1  0 -1  0  0  0
 0  0  0  0  0  0
 0  0  0  0  0  0
 0  0  0  0  0  0
 0  0  0  0  0  0
 0  1  0  0  0  0
-------------------------
Move took 9.89s, nodes visited: 67848
-------------------------
 0  0  1  0  0  0
 0  0  0  0  0  0
 0 -1 -1  0  0  0
 0  0  0  0  0  0
 0  0  0  0  0  0
 0  1  0  0  0  0
-------------------------
-------------------------
 1  0 -1  0  0  0
 0  0 -1  0  0  0
 0  0  1  0  0  0
 0  0  0  0  0  0
 0  0  0  0  0  0
 0  1  

In [5]:
numGames = 5
numWins, numTies, numLosses = 0, 0, 0
prunning = True
total_time = 0
total_nodes = 0
for i in range(numGames):
    game = PentagoGame(prunning, ui=False, print=True, depth=2)
    result = game.play()
    if result == -1:
        numWins += 1
    elif result == 0:
        numTies += 1
    else:
        numLosses += 1
    total_time += game.get_total_time()
    total_nodes += game.get_total_nodes()
    print(f"in game {i} out of {numGames} took {game.get_total_time()} seconds and {game.get_total_nodes()} nodes were visited")
print(f"{numWins} wins, {numTies} ties, {numLosses} losses, prunning {prunning}")

print(f"mean time is {total_time / numGames } seconds, mean visited nodes {total_nodes / numGames}, probility of success {numWins / numGames}")

-------------------------
 0  0  0  0  0  0
 0  0  0  0  0  0
 0  0  0  0  0  0
 0  0  0  0  0  0
 0  0  0  0  0  0
 0  0  0  0  0  0
-------------------------
-------------------------
 0  0  0  0  0  0
 0  0  0  0  0  0
 0  0  0  0  0  0
 0  0  1  0  0  0
 0  0  0  0  0  0
 0  0  0  0  0  0
-------------------------
Move took 0.88s, nodes visited: 10030
-------------------------
 0  0  0  0  0  0
 0  0  0  0  0  0
 0  0 -1  0  0  0
 0  0  0  0  0  0
 0  0  0  0  0  0
 0  0  1  0  0  0
-------------------------
-------------------------
 0  0  0  0  0  0
 0  0  0  0  0  0
 0  1 -1  0  0  0
 0  0  0  0  0  0
 0  0  0  0  0  0
 0  0  1  0  0  0
-------------------------
Move took 1.35s, nodes visited: 6898
-------------------------
 0  0 -1  0  0  0
 0  0  0  0  0  0
 0  1 -1  0  0  0
 0  0  0  0  0  0
 0  0  0  0  0  0
 1  0  0  0  0  0
-------------------------
-------------------------
 0  0 -1  0  0  0
 0  0  0  0  0  0
 0  1 -1  0  0  0
 0  0  0  1  0  0
 0  0  0  0  0  0
 1  0  0 

# 🧠 Minimax Performance at Depth 2

| Pruning         | Mean Time (s) | Mean Visited Nodes | Success Probability |
|-----------------|---------------|---------------------|----------------------|
| ❌ No Pruning   | 65.62         | 452,462.4           | 1.0                  |
| ✅ With Pruning | 7.99          | 59,182.4            | 1.0                  |


#### To play against the AI, run the cell below.

---

You can increase the AI difficulty by adjusting the `depth` parameter.  
*Note: Higher depth values result in stronger AI but slower move calculations.*


In [4]:
numGames = 1
numWins, numTies, numLosses = 0, 0, 0
prunning = True
total_time = 0
total_nodes = 0
for i in range(numGames):
    game = PentagoGame(prunning, ui=True, print=True, depth=2)
    result = game.play()
    if result == -1:
        numWins += 1
    elif result == 0:
        numTies += 1
    else:
        numLosses += 1
    total_time += game.get_total_time()
    total_nodes += game.get_total_nodes()
    print(f"in game {i} out of {numGames} took {game.get_total_time()} seconds and {game.get_total_nodes()} nodes were visited")
print(f"{numWins} wins, {numTies} ties, {numLosses} losses, prunning {prunning}")

print(f"mean time is {total_time / numGames } seconds, mean visited nodes {total_nodes / numGames}, probility of success {numWins / numGames}")

Move took 0.73s, nodes visited: 8326
Move took 1.07s, nodes visited: 5486
Move took 1.99s, nodes visited: 11647
Move took 2.68s, nodes visited: 15731
Move took 2.28s, nodes visited: 14828
Move took 1.32s, nodes visited: 7489
Move took 0.94s, nodes visited: 5025
Move took 0.68s, nodes visited: 3714
Move took 0.25s, nodes visited: 2858
Game over! Result: -1
in game 0 out of 1 took 11.939851522445679 seconds and 75104 nodes were visited
1 wins, 0 ties, 0 losses, prunning True
mean time is 11.939851522445679 seconds, mean visited nodes 75104.0, probility of success 1.0
